In [1]:
#Used in Tensorflow Model
import numpy as np
import tensorflow as tf
import tflearn
import random
from underthesea import word_tokenize
import  nltk.translate.bleu_score as bleu
from sklearn.model_selection import train_test_split
#Other
import json
import pickle
import warnings
warnings.filterwarnings("ignore")

Instructions for updating:
non-resource variables are not supported in the long term
curses is not supported on this machine (please install/reinstall curses for an optimal experience)


In [2]:
# references = [[['Chào', 'cậu.', 'lần', 'khác', 'nói', 'tiếp', 'nhé']]]
# candidates = [['Chào', 'cậu.', 'lần', 'khác', 'nói', 'tiếp', 'nhé']]
# score = bleu.corpus_bleu(references, candidates)
# print(score)

In [3]:
print("Đọc dữ liệu từ intents..........")
with open('intents2.json',encoding="utf8") as json_data:
    intents = json.load(json_data)
intents

Đọc dữ liệu từ intents..........


{'intents': [{'tag': 'loichao',
   'patterns': ['Hello',
    'xin chao',
    'Bạn khỏe không',
    'Có ai ở đó không ?',
    'Xin chào',
    'Chúc một ngày tốt lành'],
   'responses': ['Xin chào, cảm ơn bạn đã ghé thăm',
    'Rất vui được gặp lại',
    'Xin chào, tôi có thể giúp gì cho bạn  ?']},
  {'tag': 'tambiet',
   'patterns': ['Bye', 'See you later', 'Goodbye', 'tạm biệt', 'chào nhé'],
   'responses': ['Hẹn gặp lại, cảm ơn đã ghé thăm',
    'Chúc một ngày tốt lành',
    'Tạm biệt! Hãy quay lại sớm.',
    'Chào cậu. lần khác nói tiếp nhé']},
  {'tag': 'camon',
   'patterns': ['Cảm ơn', 'thanks', 'cảm ơn bạn nha'],
   'responses': ['Vui vẻ giúp đỡ!', 'Không có gì nha', 'Hân hạnh']},
  {'tag': 'Khoe',
   'patterns': ['khỏe khong',
    'Bạn có khỏe không',
    'khỏe không  ?',
    'bạn khỏe không'],
   'responses': ['Tớ vẫn khỏe, chat chit suốt ngày. hi hi']},
  {'tag': 'Thoitiet',
   'patterns': ['thời tiết hôm nay thế nào'],
   'responses': ['Không tệ', 'Trời trong xanh bao la 

In [4]:
#Xây dựng túi từ cho x và y.
words = []
classes = []
documents = []
for intent in intents['intents']:
    for pattern in intent['patterns']:
        w = word_tokenize(pattern)
        words.extend(w)
        documents.append((w, intent['tag']))
        if intent['tag'] not in classes:
            classes.append(intent['tag'])
print('End add to list')

End add to list


In [5]:
f = open('stopwords.txt', 'r', encoding='UTF-8')
data = f.read()
stop_words = data.split('\n')
ignore_words = ['?','ă', 'â', 'ê', 'ô', 'ơ', 'ư', 'ua', 'uô', 'ia', 'yê', 'iê', 'ưa', 'ươ', 'a', 'b', 'c', 'd', 'e', 'f',
                   'g', 'h', 'i', 'j', 'k', 'l', 'm', 'n', 'o', 'p', 'q', 'r', 's', 't', 'u', 'v', 'w', 'x', 'y', 'z',
                   'ph', 'th', 'tr', 'ch', 'nh', 'ng', 'kh', 'gh', 'ngh','(', ')', ',', '.', '/'] + stop_words
words = [w.lower() for w in words if w not in ignore_words]
# Xóa các từ trùng nhau trong words
words = sorted(list(set(words)))

# Xóa các từ trùng nhau trong classes
classes = sorted(list(set(classes)))
# print(ignore_words)
print (len(documents), "documents")
print (len(classes), "classes")
print (len(words), "words")
# print ("documents", documents)
# print ("classes", classes)
# print ("words", words)

223 documents
99 classes
370 words


In [6]:
# Save data x và y
data_x = []
data_y = []
for doc in documents:
    data_x.append(doc[0])
    data_y.append(doc[1])
print("End")


End


In [8]:
(X_train, X_test, y_train, y_test) = train_test_split(data_x, data_y, test_size = .3)

In [9]:
print("Tạo x và y cho Model.....")
train_x = []
train_y = []
output = []

output_empty = [0] * len(classes)

for i in range(0,len(X_train) - 1):
    bag = []
    # Lấy input(x) để tạo BOW
    pattern_words = X_train[i]
    # Chuyển đổi tất cả sang chữ thường
    pattern_words = [word.lower() for word in pattern_words]
    # Tạo BOW cho x
    for w in words:
        count = pattern_words.count(w)
        bag.append(count) if w in pattern_words else bag.append(0)                
    # Tạo BOW cho y
    output_row = list(output_empty)
    output_row[classes.index(y_train[i])] = 1

    train_x.append(bag)
    train_y.append(output_row)
print("End")

Tạo x và y cho Model.....
End


In [10]:
import keras
from keras.models import Sequential
from keras.layers import Dense, Dropout, Activation, LSTM, BatchNormalization
from tensorflow.keras.optimizers import SGD

In [11]:
XX_train, yy_train = np.array(train_x), np.array(train_y)
print(XX_train.shape)
XX_train = np.reshape(XX_train, (XX_train.shape[0], XX_train.shape[1], 1))

(155, 370)


In [12]:
print(XX_train.shape)

(155, 370, 1)


In [13]:
model = Sequential()
# model.add(LSTM(units = 64, input_shape = (XX_train.shape[1], 1)))
# model.add(Dropout(0.2))

# model.add(LSTM(units = 50, return_sequences = True ))
# model.add(Dropout(0.25))

# model.add(LSTM(units = 50, return_sequences = True))
# model.add(Dropout(0.25))

# model.add(LSTM(units = 50))
# model.add(Dropout(0.25))

model.add(LSTM(64, return_sequences=True, input_shape = (XX_train.shape[1], 1)))
model.add(BatchNormalization())

model.add(LSTM(32))
model.add(Dropout(0.5))
model.add(BatchNormalization())

# model.add(LSTM(16, activation='relu'))
# model.add(Dropout(0.2))
# model.add(BatchNormalization())

model.add(Dense(len(train_y[0]), activation='softmax'))
# model.add(Dropout(0.2))

In [16]:
# Compile model. Stochastic gradient descent with Nesterov accelerated gradient gives good results for this model
# sgd = SGD(lr=0.01, decay=1e-6, momentum=0.9, nesterov=True)
# categorical_crossentropy cosine_similarity
model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])
# model.compile(loss='mean_squared_error', optimizer='adam', metrics=['accuracy'])

In [17]:
hist = model.fit(XX_train,yy_train, epochs=300, batch_size=8, verbose=1)

Train on 155 samples
Epoch 1/300
155/155 [==============================] - 4s 28ms/sample - loss: 4.5357 - acc: 0.0194
Epoch 2/300
155/155 [==============================] - 7s 47ms/sample - loss: 4.4209 - acc: 0.0581
Epoch 3/300
155/155 [==============================] - 8s 49ms/sample - loss: 4.2810 - acc: 0.0645
Epoch 4/300
155/155 [==============================] - 6s 41ms/sample - loss: 4.2429 - acc: 0.0645
Epoch 5/300
155/155 [==============================] - 7s 44ms/sample - loss: 4.2530 - acc: 0.0710
Epoch 6/300
155/155 [==============================] - 6s 41ms/sample - loss: 4.2127 - acc: 0.0516
Epoch 7/300
155/155 [==============================] - 7s 45ms/sample - loss: 4.2024 - acc: 0.0968
Epoch 8/300
155/155 [==============================] - 7s 44ms/sample - loss: 4.1346 - acc: 0.0710
Epoch 9/300
155/155 [==============================] - 7s 43ms/sample - loss: 4.0812 - acc: 0.0581
Epoch 10/300
155/155 [==============================] - 7s 43ms/sample - loss: 4.0404 - 

155/155 [==============================] - 6s 41ms/sample - loss: 2.4899 - acc: 0.3226
Epoch 83/300
155/155 [==============================] - 7s 48ms/sample - loss: 2.6578 - acc: 0.2516
Epoch 84/300
155/155 [==============================] - 7s 43ms/sample - loss: 2.4114 - acc: 0.3355
Epoch 85/300
155/155 [==============================] - 8s 49ms/sample - loss: 2.4192 - acc: 0.3032
Epoch 86/300
155/155 [==============================] - 7s 47ms/sample - loss: 2.3459 - acc: 0.3226
Epoch 87/300
155/155 [==============================] - 7s 45ms/sample - loss: 2.3576 - acc: 0.2774
Epoch 88/300
155/155 [==============================] - 8s 49ms/sample - loss: 2.4313 - acc: 0.2839
Epoch 89/300
155/155 [==============================] - 6s 39ms/sample - loss: 2.4076 - acc: 0.3226
Epoch 90/300
155/155 [==============================] - 5s 33ms/sample - loss: 2.4545 - acc: 0.3097
Epoch 91/300
155/155 [==============================] - 6s 37ms/sample - loss: 2.3524 - acc: 0.3226
Epoch 92/300


155/155 [==============================] - 4s 28ms/sample - loss: 2.0749 - acc: 0.3742
Epoch 164/300
155/155 [==============================] - 4s 28ms/sample - loss: 1.9803 - acc: 0.3871
Epoch 165/300
155/155 [==============================] - 5s 30ms/sample - loss: 2.0363 - acc: 0.3419
Epoch 166/300
155/155 [==============================] - 4s 28ms/sample - loss: 1.8639 - acc: 0.4387
Epoch 167/300
155/155 [==============================] - 4s 28ms/sample - loss: 1.8481 - acc: 0.4194
Epoch 168/300
155/155 [==============================] - 5s 30ms/sample - loss: 1.9296 - acc: 0.3806
Epoch 169/300
155/155 [==============================] - 4s 28ms/sample - loss: 1.8845 - acc: 0.3935
Epoch 170/300
155/155 [==============================] - 4s 28ms/sample - loss: 1.7759 - acc: 0.4387
Epoch 171/300
155/155 [==============================] - 4s 29ms/sample - loss: 1.6410 - acc: 0.4968
Epoch 172/300
155/155 [==============================] - 4s 29ms/sample - loss: 1.7284 - acc: 0.4968
Epoc

155/155 [==============================] - 4s 28ms/sample - loss: 1.4423 - acc: 0.5032
Epoch 245/300
155/155 [==============================] - 4s 28ms/sample - loss: 1.4656 - acc: 0.4968
Epoch 246/300
155/155 [==============================] - 4s 29ms/sample - loss: 1.4909 - acc: 0.5355
Epoch 247/300
155/155 [==============================] - 4s 28ms/sample - loss: 1.7878 - acc: 0.4194
Epoch 248/300
155/155 [==============================] - 4s 28ms/sample - loss: 1.5264 - acc: 0.4516
Epoch 249/300
155/155 [==============================] - 4s 28ms/sample - loss: 1.3609 - acc: 0.5484
Epoch 250/300
155/155 [==============================] - 5s 30ms/sample - loss: 1.4882 - acc: 0.5290
Epoch 251/300
155/155 [==============================] - 4s 28ms/sample - loss: 1.4970 - acc: 0.5226
Epoch 252/300
155/155 [==============================] - 4s 28ms/sample - loss: 1.4602 - acc: 0.5226
Epoch 253/300
155/155 [==============================] - 5s 29ms/sample - loss: 1.3690 - acc: 0.5742
Epoc

In [18]:
def clean_up_sentence(sentence):
    #tách từ và chuyển về cùng 1 dạng
    sentence_words = word_tokenize(sentence)
    sentence_words = [word.lower() for word in sentence_words]
    return sentence_words

def bow(sentence, words):
    sentence_words = clean_up_sentence(sentence)
    bag = [0]*len(words)
    for s in sentence_words:
        for i,w in enumerate(words):
            if w == s:
                bag[i] =  bag[i] + 1
    return bag


def predict_class(sentence, model):
    p = bow(sentence, words)
    p_test = np.array([p])
    p_test = np.reshape(p_test, (p_test.shape[0], p_test.shape[1], 1))
    results = model.predict(p_test)[0]
    ERROR_THRESHOLD = 0.25
    # Lọc các xác xuất lớn hơn 0.25
    # Lưu ý: Thứ tự xác suất dựa vào thứ tự y được truyền vào
#     results = [[i,r] for i,r in enumerate(results) if r>ERROR_THRESHOLD]
    results = [[i,r] for i,r in enumerate(results)]
    # sắp xếp các xác xuất theo thứ tự giảm dần
    results.sort(key=lambda x: x[1], reverse=True)
    return_list = []
    for r in results:
        return_list.append({"intent": classes[r[0]], "probability": str(r[1])})
    return return_list

In [19]:
# predict_class('Người điều khiển xe mô tô, xe gắn máy trên đường mà trong khí thở có nồng độ cồn vượt quá bao nhiêu thì bị cấm',model)
predict_class('cảm ơn',model)

Instructions for updating:
This property should not be used in TensorFlow 2.0, as updates are applied automatically.


[{'intent': 'loichao', 'probability': '0.33224958'},
 {'intent': 'options', 'probability': '0.1333679'},
 {'intent': 'toipham_mang', 'probability': '0.096635014'},
 {'intent': 'daiphancach', 'probability': '0.0803114'},
 {'intent': 'Ten', 'probability': '0.07153492'},
 {'intent': 'tasan_thunhap', 'probability': '0.069771565'},
 {'intent': 'chongpha_nhanuoc', 'probability': '0.06395337'},
 {'intent': 'thongtin_kichdong', 'probability': '0.019754117'},
 {'intent': 'lucluongbaove_anninhmang', 'probability': '0.01932248'},
 {'intent': 'Khoe', 'probability': '0.016119549'},
 {'intent': 'camon', 'probability': '0.015619269'},
 {'intent': 'khonggian_mang', 'probability': '0.0145758465'},
 {'intent': 'anninh_mang', 'probability': '0.014064844'},
 {'intent': 'hieuluc_luatanninhmang', 'probability': '0.01221736'},
 {'intent': 'congdan_phongchongthamnhung', 'probability': '0.009605029'},
 {'intent': 'hinhphat_quydinhtrong', 'probability': '0.0072919447'},
 {'intent': 'tambiet', 'probability': '0.

In [20]:
def getResponse(ints, intents_json):
    # Lấy xác xuất đầu tiên <=> xác xuất cao nhất sau predict
    tag = ints[0]['intent']
    list_of_intents = intents_json['intents']
    for i in list_of_intents:
        if(i['tag']== tag):
            result = np.random.choice(i['responses'])
            break
    return result

def chatbot_response(msg):
    ints = predict_class(msg, model)
    if(len(ints)>0):
        res = getResponse(ints, intents)
    else:
        return 'Xin lỗi ! Ý bạn là gì ?'
    return res

In [21]:
res = chatbot_response('Cảm ơn bạn')
print(res)

Xin chào, cảm ơn bạn đã ghé thăm


In [22]:
# Lọc data x và y cho phần đánh giá
len_y_test = len(y_test)
test_X = []
test_y = []
for i in range(0,len_y_test - 1):
    if(y_test[i] in y_train):
        test_X.append(X_test[i])
        test_y.append(y_test[i])

In [23]:
# Lấy references response từ tập test
references = []
res_test = []
for y in test_y:
    found = [x for x in intents['intents'] if x["tag"] == y ][0]
    references_sub = []
    res_test.append(found['responses'])
    for response in found['responses']:
        textSplit = response.split()
        references_sub.append(textSplit)
    references.append(references_sub)
# print(references)
# print(res_test)

In [24]:
# Lấy candidates từ tập test dựa vào model
candidates = []
test_class = []
for x in test_X:
    message = ' '.join(x)
    ints = predict_class(message,model)
    if(len(ints)>0):
        tag = ints[0]['intent']
        test_class.append(tag)
        response = getResponse(ints,intents)
        candidates.append(response.split())
    else:
        candidates.append([])
        test_class.append('')
# print(candidates)

In [25]:
score = bleu.corpus_bleu(references, candidates)
print(score) 

0.10625511369037845


In [26]:
#Đánh giá phân lớp
count = 0 
for j in range(0,len(test_y) -1):
    if(test_class[j] == test_y[j]):
        count = count + 1
print(count/len(test_y))

0.14


In [23]:
# test_X
# res_test
# candidates
# Lưu kết quả test vào file json
test_json = []
len_data_test = len(test_y)
for i in range(0,len_data_test - 1):
    item = {
        "Question":' '.join(test_X[i]),
        "Answer":res_test[i],
        "Chatbot":' '.join(candidates[i])
    }
    test_json.append(item)
jsonString = json.dumps(test_json,ensure_ascii=False, indent=4)
jsonFile = open("result_test_LSTM.json", "w",encoding='utf-8')
jsonFile.write(jsonString)
jsonFile.close()

In [37]:
from tkinter import *
def send():
    #The first part, "1.0" means that the input should be read from line one, character zero (ie: the very first character).
    #END is an imported constant which is set to the string "end". The END part means to read until the end of the text box
    #is reached. The only issue with this is that it actually adds a newline to our input. 
    #So, in order to fix it we should change END to end-1c.
    #The -1c deletes 1 character, while -2c would mean delete two characters, and so on.
    msg = EntryBox.get("1.0", "end-1c").strip()
    EntryBox.delete("0.0", END)
    
    if(msg != ''):
        ChatLog.config(state=NORMAL)
        ChatLog.insert(END, "You: " + msg + '\n\n')
        ChatLog.config(foreground="#442265", font=("Verdana", 10 ))
        
        res = chatbot_response(msg)
        ChatLog.insert(END, "Bot: " + res + '\n\n')
        
        ChatLog.config(state=DISABLED)
        ChatLog.yview(END)
        
base = Tk()
base.title("Hệ thống hỏi đáp pháp luật")
base.geometry("400x500")
base.resizable(width=FALSE, height=FALSE)

#Create Chat window
ChatLog = Text(base, bd=0, bg="white", height="8", width="50", font="Arial",)

ChatLog.config(state=DISABLED)

#Bind scrollbar to Chat window
scrollbar = Scrollbar(base, command=ChatLog.yview, cursor="heart")
ChatLog['yscrollcommand'] = scrollbar.set

#Create Button to send message
SendButton = Button(base, font=("Arial",10,'bold'), text="Gửi", width="12", height=5,
                    bd=0, bg="#fca311", activebackground="#f8edeb",fg='#ffffff',
                    command= send )

#Create the box to enter message
EntryBox = Text(base, bd=0, bg="white",width="29", height="5", font="Arial")
#EntryBox.bind("<Return>", send)


#Place all components on the screen
scrollbar.place(x=376,y=6, height=386)
ChatLog.place(x=6,y=6, height=386, width=370)
EntryBox.place(x=128, y=401, height=90, width=265)
SendButton.place(x=6, y=401, height=90)

base.mainloop()
 